In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import collections
import itertools

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1611,tt0349683,120000000,203567857,King Arthur,Clive Owen|Ioan Gruffudd|Keira Knightley|Mads ...,Antoine Fuqua,Rule Your Fate.,"The story of the Arthurian legend, based on th...",126,Adventure|War|History,Jerry Bruckheimer Films|World 2000 Entertainme...,7/7/2004,6.1,2004
1759,tt0452625,25000000,59192128,Good Luck Chuck,Dane Cook|Jessica Alba|Dan Fogler|Ellia Englis...,Mark Helfrich,He has to break his curse before she breaks hi...,"Cursed since childhood, dentist Charlie Kagan ...",101,Comedy|Drama|Romance,Lionsgate|Karz Entertainment,9/21/2007,5.5,2007
294,tt1144884,40000000,186167139,The Final Destination,Bobby Campo|Shantel VanSanten|Nick Zano|Haley ...,David R. Ellis,Rest In Pieces,After a young man's premonition of a deadly ra...,82,Horror|Mystery,New Line Cinema|Practical Pictures|Parallel Zide,8/26/2009,5.5,2009
236,tt2319580,12700000,3430018,The Grand Seduction,Brendan Gleeson|Taylor Kitsch|Gordon Pinsent|L...,Don McKellar,The small town of Tickle Cove needed a doctor....,A small fishing village must procure a local d...,112,Comedy,Max Films Productions|Morag Loves Company,5/30/2014,6.7,2014
135,tt1956620,40000000,126069509,Sex Tape,Cameron Diaz|Jason Segel|Rob Corddry|Ellie Kem...,Jake Kasdan,A movie about a movie they don't want you to see.,"When Jay and Annie first got together, their r...",97,Comedy,Escape Artists|Media Rights Capital|Sony Pictu...,7/17/2014,5.3,2014


In [3]:
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [4]:
answers = {} # создадим словарь для ответов

# предобработки колонок:

data['profit'] = data['revenue'] - data['budget'] # создание колонки с прибылью
data['genres'] = data['genres'].apply(lambda x: x.split('|')) 
data['director'] = data['director'].apply(lambda x: x.split('|')) 
data['cast'] = data['cast'].apply(lambda x: x.split('|')) 
data['production_companies'] = data['production_companies'].apply(lambda x: x.split('|')) 
data['release_date'] = data['release_date'].apply(lambda x: x.split('/')) 
data['release_month'] = data['release_date'].apply(lambda x: x[0]) # создание колонки с месяцем релиза
data['lenght_title'] = data['original_title'].apply(lambda x: len(x)) # создание колонки с длиной названия
data['lenght_split_overview'] = data['overview'].apply(lambda x: len(x.split(' '))) #создание колонки с количеством 
                                                                                    #слов в описании
data.sample(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year,profit,release_month,lenght_title,lenght_split_overview
932,tt0299658,45000000,306776732,Chicago,"[Catherine Zeta-Jones, RenÃ©e Zellweger, Queen...",[Rob Marshall],"If You Can't Be Famous, Be Infamous.",Murderesses Velma Kelly (a chanteuse and tease...,113,"[Action, Comedy, Crime, Drama, Music]","[Miramax Films, Producers Circle, Storyline En...","[12, 26, 2002]",6.5,2002,261776732,12,7,58
96,tt1741273,19500000,32189727,Secret in Their Eyes,"[Chiwetel Ejiofor, Nicole Kidman, Julia Robert...",[Billy Ray],Don't look back,"A tight-knit team of FBI investigators, along ...",111,"[Crime, Drama, Mystery]","[Ingenious Media, Gran Via Productions, IM Glo...","[10, 14, 2015]",6.2,2015,12689727,10,20,30
1353,tt0425123,58000000,102854431,Just Like Heaven,"[Reese Witherspoon, Mark Ruffalo, Donal Logue,...",[Mark Waters],It's a wonderful afterlife.,Shortly after David Abbott moves into his new ...,95,"[Comedy, Fantasy, Romance]","[DreamWorks SKG, MacDonald/Parkes Productions]","[9, 16, 2005]",6.5,2005,44854431,9,16,52
338,tt1013752,85000000,363164265,Fast & Furious,"[Vin Diesel, Paul Walker, Jordana Brewster, Mi...",[Justin Lin],New Model. Original Parts.,"When a crime brings them back to L.A., fugitiv...",107,"[Action, Crime, Drama, Thriller]","[Universal Pictures, Original Film, One Race P...","[4, 2, 2009]",6.4,2009,278164265,4,14,67
1432,tt0382628,30000000,25473093,Dark Water,"[Jennifer Connelly, John C. Reilly, Tim Roth, ...",[Walter Salles],"This Season, The Mystery Of The Darkness Will ...",Dahlia Williams and her daughter Cecelia move ...,105,"[Drama, Horror, Thriller]","[Vertigo Entertainment, Touchstone Pictures, P...","[6, 27, 2005]",5.2,2005,-4526907,6,10,62


# 1. У какого фильма из списка самый большой бюджет?

In [5]:
data[data['budget'] == data['budget'].max()][['imdb_id', 'original_title', 'budget']]

,imdb_id,original_title,budget
723,tt1298650,Pirates of the Caribbean: On Stranger Tides,380000000


In [6]:
answers['1'] = '5. Pirates of the Caribbean: On Stranger Tides (tt1298650)', '+' 

# 2. Какой из фильмов самый длительный (в минутах)?

In [7]:
data[data['runtime'] == data['runtime'].max()][['imdb_id', 'original_title', 'runtime']]

,imdb_id,original_title,runtime
1157,tt0279111,Gods and Generals,214


In [8]:
answers['2'] = '2. Gods and Generals (tt0279111)', '+'

# 3. Какой из фильмов самый короткий (в минутах)?





In [9]:
data[data['runtime'] == data['runtime'].min()][['imdb_id', 'original_title', 'runtime']]

,imdb_id,original_title,runtime
768,tt1449283,Winnie the Pooh,63


In [10]:
answers['3'] = '3. Winnie the Pooh (tt1449283)', '+'

# 4. Какова средняя длительность фильмов?


In [11]:
round(data['runtime'].mean())

110

In [12]:
answers['4'] = '2. 110', '+'

# 5. Каково медианное значение длительности фильмов? 

In [13]:
data['runtime'].median()

107.0

In [14]:
answers['5'] = '1. 107', '+'

# 6. Какой самый прибыльный фильм?

In [15]:
data[data['profit'] == data['profit'].max()][['imdb_id', 'original_title', 'profit']]

,imdb_id,original_title,profit
239,tt0499549,Avatar,2544505847


In [16]:
answers['6'] = '5. Avatar (tt0499549)', '+'

# 7. Какой фильм самый убыточный? 

In [17]:
data[data['profit'] == data['profit'].min()][['imdb_id', 'original_title', 'profit']]

,imdb_id,original_title,profit
1245,tt1210819,The Lone Ranger,-165710090


In [18]:
answers['7'] = '5. The Lone Ranger (tt1210819)', '+'

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [19]:
len(data[data['revenue'] > data['budget']])

1478

In [20]:
answers['8'] = '1. 1478', '+'

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [21]:
#2 вариант: отсортировать data по 2008 году, сохранить новый датасет, и в нем найти revenue.max(), как в 10 вопросе. 
data.loc[data['release_year'] == 2008][['imdb_id', 'original_title', 'revenue']].sort_values(['revenue'], 
                                                                                 ascending=False).head(2)

,imdb_id,original_title,revenue
599,tt0468569,The Dark Knight,1001921825
603,tt0367882,Indiana Jones and the Kingdom of the Crystal S...,786636033


In [22]:
answers['9'] = '4. The Dark Knight (tt0468569)', '+'

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [23]:
#2 вариант: сделать все в одну строку, как в 9 вопросе. 
data2012_2014 = data.loc[(data['release_year'] > 2011)&(data['release_year'] < 2015)]
data2012_2014[data2012_2014['profit'] == data2012_2014['profit'].min()][['imdb_id', 'original_title', 'profit']]

,imdb_id,original_title,profit
1245,tt1210819,The Lone Ranger,-165710090


In [24]:
answers['10'] = '5. The Lone Ranger (tt1210819)', '+'

# 11. Какого жанра фильмов больше всего?

In [25]:
genre_count = collections.Counter() 

for row in data['genres']:
    for genre in row:
        genre_count[genre] += 1
        
genre_count.most_common() 

[('Drama', 782),
 ('Comedy', 683),
 ('Thriller', 596),
 ('Action', 582),
 ('Adventure', 415),
 ('Crime', 315),
 ('Romance', 308),
 ('Family', 260),
 ('Science Fiction', 248),
 ('Fantasy', 222),
 ('Horror', 176),
 ('Mystery', 168),
 ('Animation', 139),
 ('Music', 64),
 ('History', 62),
 ('War', 58),
 ('Western', 19),
 ('Documentary', 8),
 ('Foreign', 2)]

2 вариант.

In [26]:
data_exp_genres = data.explode('genres')
data_exp_genres['genres'].value_counts()

Drama              782
Comedy             683
Thriller           596
Action             582
Adventure          415
Crime              315
Romance            308
Family             260
Science Fiction    248
Fantasy            222
Horror             176
Mystery            168
Animation          139
Music               64
History             62
War                 58
Western             19
Documentary          8
Foreign              2
Name: genres, dtype: int64

In [27]:
answers['11'] = '3. Drama (782)', '+'

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [28]:
genre_count = collections.Counter() 

for row in data[data['profit'] > 0]['genres']:
    for genre in row:
        genre_count[genre] += 1
        
genre_count.most_common()

[('Drama', 560),
 ('Comedy', 551),
 ('Thriller', 446),
 ('Action', 444),
 ('Adventure', 337),
 ('Romance', 242),
 ('Crime', 231),
 ('Family', 226),
 ('Science Fiction', 195),
 ('Fantasy', 188),
 ('Horror', 150),
 ('Animation', 120),
 ('Mystery', 119),
 ('Music', 47),
 ('History', 46),
 ('War', 41),
 ('Western', 12),
 ('Documentary', 7)]

2 вариант. 

In [29]:
data_exp_genres[data_exp_genres['profit'] > 0]['genres'].value_counts()

Drama              560
Comedy             551
Thriller           446
Action             444
Adventure          337
Romance            242
Crime              231
Family             226
Science Fiction    195
Fantasy            188
Horror             150
Animation          120
Mystery            119
Music               47
History             46
War                 41
Western             12
Documentary          7
Name: genres, dtype: int64

In [30]:
answers['12'] = '1. Drama (560)', '+'

# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [31]:
data_exp_director = data.explode('director')
data_exp_director.groupby(['director'])['revenue'].sum().sort_values(ascending=False)

director
Peter Jackson        6490593685
Christopher Nolan    4167548502
David Yates          4154295625
Michael Bay          3886938960
J.J. Abrams          3579169916
                        ...    
David MichÃ´d           2295423
Steven Shainberg        2281089
Paul Schrader           2062066
Keanu Reeves            2054941
Simon Hunter            2033165
Name: revenue, Length: 997, dtype: int64

In [32]:
answers['13'] = '5. Peter Jackson (6490593685)', '+'

# 14. Какой режисер снял больше всего фильмов в стиле Action?

In [33]:
data_exp_genres_director = data.explode('genres').explode('director')
data_exp_genres_director[data_exp_genres_director['genres'] == 'Action']['director'].value_counts().sort_values(
                                                                                                    ascending=False)

Robert Rodriguez      9
Paul W.S. Anderson    7
Michael Bay           7
Ridley Scott          6
Antoine Fuqua         6
                     ..
John Carpenter        1
Joseph Kahn           1
Don Bluth             1
Kerry Conran          1
Scott Stewart         1
Name: director, Length: 364, dtype: int64

2 вариант.

In [34]:
director_genres = data_exp_genres_director.pivot_table(index='director', columns='genres', values='original_title', 
                                                       aggfunc='count', fill_value=0)
director_genres['Action'].sort_values(ascending=False)

director
Robert Rodriguez      9
Paul W.S. Anderson    7
Michael Bay           7
Ridley Scott          6
Antoine Fuqua         6
                     ..
Mark Herman           0
Mark Helfrich         0
Mark Duplass          0
Mark Dindal           0
Jorge Blanco          0
Name: Action, Length: 997, dtype: int64

In [35]:
answers['14'] = '3. Robert Rodriguez (9)', '+'

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [36]:
data_exp_cast = data.explode('cast')
data_exp_cast[data_exp_cast['release_year'] == 2012].groupby(['cast'])['revenue'].sum().sort_values(ascending=False)

cast
Chris Hemsworth      2027450773
Denis Leary          1629460639
Anne Hathaway        1522851057
Chris Evans          1519557910
Robert Downey Jr.    1519557910
                        ...    
Jason Bateman           3428048
Danny Huston            2106557
Sami Gayle              2106557
Josh Lucas              2106557
Nicolas Cage            2106557
Name: revenue, Length: 466, dtype: int64

In [37]:
answers['15'] = '3. Chris Hemsworth (2027450773)', '+'

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [38]:
data_exp_cast[data_exp_cast['budget'] > data_exp_cast['budget'].mean()]['cast'].value_counts().sort_values(
                                                                                                ascending=False)

Matt Damon           18
Adam Sandler         17
Angelina Jolie       16
Samuel L. Jackson    15
Tom Cruise           15
                     ..
Jon Hamm              1
Rachael Taylor        1
Sam Robards           1
Madeleine Stowe       1
Cillian Murphy        1
Name: cast, Length: 1505, dtype: int64

In [39]:
answers['16'] = '3. Matt Damon (18)', '+'

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [40]:
data_exp_cast_genres = data.explode('cast').explode('genres')
data_exp_cast_genres[data_exp_cast_genres['cast'] == 'Nicolas Cage']['genres'].value_counts().sort_values(
                                                                                              ascending=False)

Action             17
Thriller           15
Drama              12
Crime              10
Fantasy             8
Adventure           7
Comedy              6
Science Fiction     4
Mystery             3
Animation           3
Family              3
History             2
Romance             1
Horror              1
War                 1
Name: genres, dtype: int64

In [41]:
answers['17'] = '2. Action (17)', '+'

# 18. Самый убыточный фильм от Paramount Pictures

In [42]:
data_exp_prod_companies = data.explode('production_companies')
data_exp_prod_companies[data_exp_prod_companies['production_companies'] == 'Paramount Pictures'][['imdb_id', 
                                                                  'original_title', 'profit']].sort_values('profit')

,imdb_id,original_title,profit
925,tt0267626,K-19: The Widowmaker,-64831034
1123,tt0300556,Timeline,-60519261
1722,tt0435705,Next,-51788987
1623,tt0375173,Alfie,-46604061
1631,tt0315297,Twisted,-24805000
...,...,...,...
730,tt1229238,Mission: Impossible - Ghost Protocol,549713380
1738,tt0418279,Transformers,559709780
603,tt0367882,Indiana Jones and the Kingdom of the Crystal S...,601636033
317,tt1055369,Transformers: Revenge of the Fallen,686297228


In [43]:
answers['18'] = '1. K-19: The Widowmaker (tt0267626). -64831034', '+'

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [44]:
data.groupby(['release_year'])['revenue'].sum().sort_values(ascending=False)

release_year
2015    25449202382
2014    23405862953
2013    23213799791
2012    23079001687
2011    22676791872
2010    21071204961
2009    20261791024
2008    18252781990
2007    18162406801
2004    15663430720
2005    15309425558
2006    14775042320
2003    14346123312
2002    14136361487
2001    13017764865
2000    10664099805
Name: revenue, dtype: int64

In [45]:
answers['19'] = '5. 2015 25449202382', '+'

# 20. Какой самый прибыльный год для студии Warner Bros?

In [46]:
#выясняем названия компаний Warner Bros, т.к. в data нет компании 'Warner Bros'
companies = list(data_exp_prod_companies['production_companies'].unique())
for element in companies:
    if 'Warner Bros' in element:
        print(element)

Warner Bros.
Warner Bros. Animation
Warner Bros. Interactive Entertainment
Warner Bros. Pictures


In [47]:
warner_bros = data_exp_prod_companies[data_exp_prod_companies['production_companies'].str.contains('Warner Bros')]
warner_bros.groupby('release_year')['profit'].sum().sort_values(ascending=False)

release_year
2014    2292949646
2007    2201675217
2008    2134595031
2010    1974712985
2011    1871393682
2003    1844008221
2009    1822454136
2013    1636453400
2004    1631933725
2005    1551980298
2001    1282142579
2012    1258020056
2002    1022709901
2015     870368348
2006     625351872
2000     452631386
Name: profit, dtype: int64

In [48]:
answers['20'] = '1. 2014 2292949646', '+'

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [49]:
data['release_month'].value_counts()

9     227
12    190
10    186
8     161
3     156
4     149
6     147
11    146
7     142
5     140
2     135
1     110
Name: release_month, dtype: int64

In [50]:
answers['21'] = '4. Сентябрь 227', '+'

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [51]:
len(data[data['release_month'].isin(['6', '7', '8'])])

450

2 вариант.

In [52]:
data[data['release_month'].isin(['6', '7', '8'])]['original_title'].count()

450

In [53]:
answers['22'] = '2. 450', '+'

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [54]:
winter = data_exp_director[data_exp_director['release_month'].isin(['11', '12', '1'])]
winter.groupby(['director'])['original_title'].count().sort_values(ascending=False)

director
Peter Jackson          8
Clint Eastwood         7
Steven Soderbergh      6
Martin Scorsese        4
Peter Farrelly         4
                      ..
Michel Gondry          1
Miguel Arteta          1
Mike Binder            1
Mike Mitchell          1
Julie Anne Robinson    1
Name: original_title, Length: 371, dtype: int64

In [55]:
answers['23'] = '5. Peter Jackson 8', '+'

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [56]:
data_exp_prod_companies.groupby('production_companies')['lenght_title'].mean().sort_values(ascending=False)

production_companies
Four By Two Productions       83.0
Jim Henson Company, The       59.0
Dos Corazones                 47.0
Museum Canada Productions     46.0
Polsky Films                  46.0
                              ... 
Everest Entertainment          3.0
Berlanti Productions           3.0
XM2 Productions                2.0
Ixtlan Productions             2.0
Global Entertainment Group     2.0
Name: lenght_title, Length: 1771, dtype: float64

In [57]:
answers['24'] = '5. Four By Two Productions 83', '+'

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [58]:
grouped_data = data_exp_prod_companies.groupby(['production_companies']).agg({'original_title': 'count', 
                                                                   'lenght_split_overview': 'sum'})

grouped_data['middle'] = grouped_data['lenght_split_overview']/grouped_data['original_title']
grouped_data['middle'].sort_values(ascending=False)

production_companies
Midnight Picture Show                    175.0
Room 9 Entertainment                     161.0
Heineken Branded Entertainment           161.0
98 MPH Productions                       159.0
Brookwell-McNamara Entertainment         156.0
                                         ...  
London Boulevard                          13.0
Phantom Four                              13.0
Henceforth                                13.0
Empire Pictures                           11.0
Motion Picture Corporation of America     11.0
Name: middle, Length: 1771, dtype: float64

In [59]:
answers['25'] = '3. Midnight Picture Show 175', '+'

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [60]:
percent = round(len(data)*0.01)
percent

19

In [61]:
data.sort_values('vote_average', ascending=False)[:19][['original_title', 'vote_average']]

,original_title,vote_average
599,The Dark Knight,8.1
118,Interstellar,8.0
125,The Imitation Game,8.0
9,Inside Out,8.0
34,Room,8.0
1183,The Wolf of Wall Street,7.9
128,Gone Girl,7.9
1191,12 Years a Slave,7.9
119,Guardians of the Galaxy,7.9
1081,The Lord of the Rings: The Return of the King,7.9


2 вариант.

In [62]:
data['vote_average'].quantile(q=0.99)

7.8

In [63]:
data[data['vote_average'] > 7.8][['original_title', 'vote_average']]

,original_title,vote_average
9,Inside Out,8.0
34,Room,8.0
118,Interstellar,8.0
119,Guardians of the Galaxy,7.9
125,The Imitation Game,8.0
128,Gone Girl,7.9
138,The Grand Budapest Hotel,7.9
370,Inception,7.9
599,The Dark Knight,8.1
872,The Pianist,7.9


In [64]:
answers['26'] = '1. Inside Out, The Dark Knight, 12 Years a Slave', '+'

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


In [65]:
list_cast = list(data['cast'])
list_cast[:2]

[['Chris Pratt',
  'Bryce Dallas Howard',
  'Irrfan Khan',
  "Vincent D'Onofrio",
  'Nick Robinson'],
 ['Tom Hardy',
  'Charlize Theron',
  'Hugh Keays-Byrne',
  'Nicholas Hoult',
  'Josh Helman']]

In [66]:
list_pair_cast = []

for element in list_cast:
    pair = list(itertools.combinations(element, 2))
    list_pair_cast.append(pair)
    
list_pair_cast[:2]

[[('Chris Pratt', 'Bryce Dallas Howard'),
  ('Chris Pratt', 'Irrfan Khan'),
  ('Chris Pratt', "Vincent D'Onofrio"),
  ('Chris Pratt', 'Nick Robinson'),
  ('Bryce Dallas Howard', 'Irrfan Khan'),
  ('Bryce Dallas Howard', "Vincent D'Onofrio"),
  ('Bryce Dallas Howard', 'Nick Robinson'),
  ('Irrfan Khan', "Vincent D'Onofrio"),
  ('Irrfan Khan', 'Nick Robinson'),
  ("Vincent D'Onofrio", 'Nick Robinson')],
 [('Tom Hardy', 'Charlize Theron'),
  ('Tom Hardy', 'Hugh Keays-Byrne'),
  ('Tom Hardy', 'Nicholas Hoult'),
  ('Tom Hardy', 'Josh Helman'),
  ('Charlize Theron', 'Hugh Keays-Byrne'),
  ('Charlize Theron', 'Nicholas Hoult'),
  ('Charlize Theron', 'Josh Helman'),
  ('Hugh Keays-Byrne', 'Nicholas Hoult'),
  ('Hugh Keays-Byrne', 'Josh Helman'),
  ('Nicholas Hoult', 'Josh Helman')]]

In [67]:
dict_pair_cast = collections.Counter()

for element in list_pair_cast:
    for pairs in element:
        dict_pair_cast[pairs] += 1
    
dict_pair_cast.most_common(3)

[(('Daniel Radcliffe', 'Rupert Grint'), 8),
 (('Daniel Radcliffe', 'Emma Watson'), 8),
 (('Rupert Grint', 'Emma Watson'), 7)]

In [68]:
answers['27'] = '5. Daniel Radcliffe & Rupert Grint, 8', '+'

# Submission

In [69]:
# ответы к каждому вопросу
answers

{'1': ('5. Pirates of the Caribbean: On Stranger Tides (tt1298650)', '+'),
 '2': ('2. Gods and Generals (tt0279111)', '+'),
 '3': ('3. Winnie the Pooh (tt1449283)', '+'),
 '4': ('2. 110', '+'),
 '5': ('1. 107', '+'),
 '6': ('5. Avatar (tt0499549)', '+'),
 '7': ('5. The Lone Ranger (tt1210819)', '+'),
 '8': ('1. 1478', '+'),
 '9': ('4. The Dark Knight (tt0468569)', '+'),
 '10': ('5. The Lone Ranger (tt1210819)', '+'),
 '11': ('3. Drama (782)', '+'),
 '12': ('1. Drama (560)', '+'),
 '13': ('5. Peter Jackson (6490593685)', '+'),
 '14': ('3. Robert Rodriguez (9)', '+'),
 '15': ('3. Chris Hemsworth (2027450773)', '+'),
 '16': ('3. Matt Damon (18)', '+'),
 '17': ('2. Action (17)', '+'),
 '18': ('1. K-19: The Widowmaker (tt0267626). -64831034', '+'),
 '19': ('5. 2015 25449202382', '+'),
 '20': ('1. 2014 2292949646', '+'),
 '21': ('4. Сентябрь 227', '+'),
 '22': ('2. 450', '+'),
 '23': ('5. Peter Jackson 8', '+'),
 '24': ('5. Four By Two Productions 83', '+'),
 '25': ('3. Midnight Picture Show

In [70]:
len(answers)

27